In [1]:
# We are using CEDAR-FOX for the simple machine learning approch
# But we found that batch verification is not working for more than 3 pair at a time
# So we wrote some python code to automate some of that process

In [2]:
import shutil
from pathlib import Path
import pandas as pd
import csv
import numpy as np

In [3]:
# The following function splits the data into folders with 6 images each
# It also creates a txt file in each directory with 3 pairs
def Split_Images():
    i = 0 
    for i in range(75):
        j=0
        if not os.path.exists(".\Folder"+str(i)): 
            os.mkdir(".\Folder"+str(i))
            #file= open("./Folder"+str(i)+"/filenames.txt","w")
            Path("./Folder"+str(i)+"/filenames.txt").touch()
            
        for j in range(3):
            img0_tuple = random.choice(folder_dataset.imgs)
            should_get_same_class = random.randint(0,1) 
            if should_get_same_class:
                while True:
                    img1_tuple = random.choice(folder_dataset.imgs) 
                    if img0_tuple[1]==img1_tuple[1] :
                        break
            else:
                img1_tuple = random.choice(folder_dataset.imgs)
             
            shutil.copy2(img0_tuple[0], "./Folder"+str(i) )
            shutil.copy2(img1_tuple[0], "./Folder"+str(i) )
            
            file= open("./Folder"+str(i)+"/filenames.txt","a")
            file.write(img0_tuple[0][-14:])
            file.write(" ")
            file.write(img1_tuple[0][-14:] + "\n") 
            file.close()
            j = j + 1
        i = i + 1 
    return 0

In [4]:
# use the txt files generated by the Split_Images() method
# and run it through batch verification in CEDAR-FOX
# this will spit out .xsl files with the results

# in these xsl files, the 'Final' column has either a positive or negative value
# the similarity is judged based on this value

In [5]:
# this should be run after CEDAR-FOX part is completed and .xsl files are generated
# this piece of code goes through the folders created by Split_Images() method
# it evaluates a 0 or 1 based on the 'Final' column
# it also evalues the ground ground_truth_labels, which in this case is obtained by looking at the name of the author

i=0
predicted_labels = []
ground_truth_labels = []
exists = False
opened = False
parsed = False
for i in range(64):
    if i == 11:
        continue
    try:
        file_name = "C:/Users/ashwi/Downloads/CEDAR Fox Folder Set/Folder"+str(i)+"/filenames.out.xls"
        #print(i)
        exists = True
    except:
        exists = False
        print("nothing to see here")
        continue
    if(exists):
        try:
            ifile  = open(file_name,"rt")
            opened = True
        except:
            opened = False
            print("nothing to open here: " + file_name)
            continue
        if(opened):
            try:
                rows = list(csv.reader(ifile, delimiter='\t'))
                parsed = True
            except:
                print("nothing to parses here")
                parsed = False
                continue
            if(parsed):
                if(len(rows) > 1):
                    del rows[0]
                    for row in rows:
                        if float(row[2]) > 0:
                            predicted_labels.append(0)
                        else:
                            predicted_labels.append(1)
                        if row[0] == row[1]:
                            ground_truth_labels.append(0)
                        else:
                            ground_truth_labels.append(1)

In [6]:
CEDAR_matches = np.equal(predicted_labels, ground_truth_labels)
n = 0
for i in CEDAR_matches:
    if(i):
        n = n + 1
print("Accuracy: " + str(n*100 / len(ground_truth_labels))+"%")

Accuracy: 84.31372549019608%


In [7]:
# Getting an accuracy of 84% for 100 samples. Compared to 78% of the deep learning approach. But there is a caveat.
# This is not a perfect comparison to the deep learning approach as that test is being made there over ~3000 samples